In [39]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf
from datetime import datetime
from datetime import timedelta

In [40]:
def fetch_stocks():
    filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    # filters = ['sh_avgvol_o2000', 'ind_stocksonly', 'cap_largeover', 'sh_price_o10']
    
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [41]:
def cal_VRRS(ticker, ref, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * 100

In [42]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = 100 * ((close - highh) / (highh - lowl))
    return wr

In [43]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        if len(x) > 1:
            return float(x.replace('B', '')) * 1000000000
        return 1000000000.0
    if 'T' in x:
        if len(x) > 1:
            return float(x.replace('T', '')) * 1000000000000
        return 1000000000000000.0

    return 0.0

In [44]:
ticker_dict = {}

In [45]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.57it/s]


In [ ]:
start_date = "2017-01-01"
end_date = "2022-07-09"

interval_time = '1d'

end_date = datetime.today().date()

spy = yf.Ticker("SPY")
spy = spy.history(start=start_date, end=end_date, interval=interval_time)['Close'].dropna()

for stock in stock_list:
    
    ticker = stock['Ticker']
    
    # avg_volume = value_to_float(stock['Avg Volume'])
    # if avg_volume > 2000000:

    stock = yf.Ticker(ticker)    

    stock_close = stock.history(start=start_date, end=end_date, interval=interval_time)['Close'].dropna()
    stock_now = stock.history(start=start_date, end=end_date, interval=interval_time).dropna()
    
    williams = get_wr(stock_now['High'], stock_now['Low'], stock_now['Close'], 2).iloc[-1]

    # Calculate the VRRS indicator:
    rolling_length = 21 

    vrrs = cal_VRRS(stock_close, spy, rolling_length).iloc[-1]
    week_ago_VRRS = cal_VRRS(stock_close.iloc[:-1], spy.iloc[:-1], rolling_length).iloc[-1]

    difference = float(vrrs) - float(week_ago_VRRS)

    if abs(difference) > 2 and williams >= -20 and vrrs > 0:
        print(ticker, ' || VRRS: ', round(vrrs, 2), '|| Week_Ago_VRRS: ', round(week_ago_VRRS, 2), ' || ', round(difference, 2))
    elif abs(difference) > 2 and williams <= -80 and vrrs < 0:
        print(ticker, ' || VRRS: ', round(vrrs, 2), '|| Week_Ago_VRRS: ', round(week_ago_VRRS, 2), ' || ', round(difference, 2))


LABU  || VRRS:  44.7 || Week_Ago_VRRS:  42.68  ||  2.02
TMF  || VRRS:  -3.29 || Week_Ago_VRRS:  0.06  ||  -3.36
UVIX  || VRRS:  -17.87 || Week_Ago_VRRS:  -13.71  ||  -4.16
SOXS  || VRRS:  -2.6 || Week_Ago_VRRS:  0.8  ||  -3.4
